In [1]:
import geopandas
import scipy.io
import pandas as pd
from EchoPro import EchoPro
epro_2019 = EchoPro(init_file_path='./config_files/initialization_config.yml',
                    survey_year_file_path='./config_files/survey_year_2019_config.yml',
                    source=3,
                    bio_data_type=1,
                    age_data_status=1, 
                    exclude_age1=True)

A check of the initialization file needs to be done!
A check of the survey year file needs to be done!
Loading biological data ...


/Users/brandonreyes/UW_work/EchoPro_work/EchoPro/EchoPro/echo_pro.py:273: UserWarning: We are currently using wgt_total from Matlab for CV, change this!
  warnings.warn("We are currently using wgt_total from Matlab for CV, change this!")


In [2]:
krig_mesh = epro_2019.get_kriging_mesh()
# fmap = krig_mesh.get_folium_map()  # this function is not necessary, keeping it here as a reminder 

In [3]:
# get geopandas representation of final_biomass_table
# allows us to plot transect data
df = epro_2019.final_biomass_table
gdf = geopandas.GeoDataFrame(df, 
                             geometry=geopandas.points_from_xy(df.Longitude, df.Latitude))

# reseting the index so that we can select data based on column name
gdf = gdf.reset_index()

In [9]:
# lat_INPFC = [np.NINF, 36, 40.5, 43.000, 45.7667, 48.5, 55.0000]  # INPFC
# cva = epro_2019.run_cv_analysis(lat_INPFC, kriged_data=False)
# 0.13282819671923227